# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Jaclyn Bommarito</p>
### <p style="text-align: right;"> &#9989; jaclynbommarito</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

git clone https://github.com/jaclynbommarito/CMSE202-s23-turnin/tree/hw4

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [1]:
import pandas as pd
import statsmodels.api as sm 
from sklearn.svm import SVC, LinearSVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay,classification_report

In [2]:
#Write your code here
weather = pd.read_csv("aussie_weather_data.csv")
weather

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009,1,1,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,...,84.0,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0
1,2009,1,2,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,...,79.0,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0
2,2009,1,3,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,...,87.0,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1
3,2009,1,4,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,...,92.0,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1
4,2009,1,5,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,...,86.0,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39510,2017,6,20,Darwin,19.3,33.4,0.0,6.0,11.0,22.5,...,63.0,32.0,1013.9,1010.5,0.0,1.0,24.5,32.3,0,0
39511,2017,6,21,Darwin,21.2,32.6,0.0,7.6,8.6,0.0,...,56.0,28.0,1014.6,1011.2,7.0,0.0,24.8,32.0,0,0
39512,2017,6,22,Darwin,20.7,32.8,0.0,5.6,11.0,0.0,...,46.0,23.0,1015.3,1011.8,0.0,0.0,24.8,32.1,0,0
39513,2017,6,23,Darwin,19.5,31.8,0.0,6.2,10.6,337.5,...,62.0,58.0,1014.9,1010.7,1.0,1.0,24.8,29.2,0,0


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [3]:
#Write your code here
weather["Location"].unique()

array(['NorfolkIsland', 'SydneyAirport', 'WaggaWagga', 'MelbourneAirport',
       'Mildura', 'Watsonia', 'Brisbane', 'Cairns', 'Townsville',
       'MountGambier', 'Nuriootpa', 'PerthAirport', 'Perth',
       'AliceSprings', 'Darwin'], dtype=object)

#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [4]:
#Write your code here
weather1 = weather[weather["Location"] == "Mildura"]
weather1 = weather1.drop(columns=["Location"])

#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [11]:
#Write your code here
X = weather1.drop(columns=["WindGustSpeed"])
y = weather1["WindGustSpeed"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
X_train

,Year,Month,DayOfMonth,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
12880,2015,10,17,17.2,23.8,0.1,6.3,4.1,247.5,247.5,...,64.0,83.0,1025.1,1025.2,8.0,6.0,17.4,15.2,0,1
11759,2012,3,12,11.6,28.4,0.0,5.4,11.4,0.0,0.0,...,64.0,29.0,1019.4,1016.9,1.0,1.0,16.1,27.3,0,0
11841,2012,6,6,2.7,14.8,0.0,2.8,9.0,315.0,270.0,...,83.0,42.0,1024.1,1022.7,0.0,1.0,5.7,14.4,0,0
12031,2013,3,18,8.0,24.8,0.0,8.0,11.3,292.5,292.5,...,55.0,25.0,1024.0,1021.0,1.0,1.0,12.7,22.9,0,0
11469,2011,5,17,0.0,16.3,0.0,1.8,9.1,45.0,67.5,...,81.0,42.0,1032.5,1028.5,0.0,2.0,7.2,15.8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11639,2011,11,10,12.0,25.2,3.6,6.4,12.8,157.5,202.5,...,64.0,26.0,1018.2,1018.2,1.0,1.0,16.6,23.9,1,0
11584,2011,9,15,3.1,26.5,0.0,5.6,9.8,135.0,45.0,...,56.0,21.0,1024.9,1019.7,1.0,2.0,13.3,24.8,0,0
11775,2012,3,28,14.5,27.8,0.0,4.0,8.6,90.0,45.0,...,66.0,32.0,1021.1,1018.4,5.0,5.0,16.2,26.3,0,0
10914,2009,9,2,7.8,23.7,0.0,3.4,9.4,90.0,45.0,...,45.0,19.0,1019.8,1013.5,1.0,1.0,15.9,23.1,0,0


## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

*Since we want to predict a continuous variable (WindGustSpeed), linear regression would be more appropriate for this task. Logistic regression is used for predicting categorical variables, particularly binary outcomes.*

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [12]:
#Write your code here
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

model = sm.OLS(y_train, X_train).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          WindGustSpeed   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.642
Method:                 Least Squares   F-statistic:                     163.0
Date:                Fri, 21 Apr 2023   Prob (F-statistic):               0.00
Time:                        16:36:31   Log-Likelihood:                -6982.9
No. Observations:                2075   AIC:                         1.401e+04
Df Residuals:                    2051   BIC:                         1.415e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          118.5046    137.004      0.865   

*Statistically significant features on WindGustSpeed: 
WindGustSpeed
MinTemp
MaxTemp
Rainfall
Sunshine
Humidity3pm
Pressure3pm
Temp3pm
RainToday

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [13]:
#Write your code here
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype('int')
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))
print("\nClassification Report: \n", classification_report(y_test, y_pred))

Confusion matrix: 
 [[0 0 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 [7 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [3 0 0 ... 0 0 0]]

Classification Report: 
               precision    recall  f1-score   support

         1.0       0.00      0.00      0.00       0.0
        15.0       0.00      0.00      0.00       2.0
        17.0       0.00      0.00      0.00       7.0
        19.0       0.00      0.00      0.00       5.0
        20.0       0.00      0.00      0.00      14.0
        22.0       0.00      0.00      0.00      14.0
        24.0       0.00      0.00      0.00      24.0
        26.0       0.00      0.00      0.00      27.0
        28.0       0.00      0.00      0.00      41.0
        30.0       0.00      0.00      0.00      36.0
        31.0       0.00      0.00      0.00      36.0
        33.0       0.00      0.00      0.00      38.0
        35.0       0.00      0.00      0.00      29.0
        37.0       0.00      0.00      0.00      24.0
        39.0       0.00

/home/bommar70/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bommar70/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bommar70/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bommar70/.local/lib/python3.8/site-p

#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

*After fitting the test data and creating a confusion matrix, the model has a higher precision score than recall, meaning that it performed better at correctly predicting positive cases than identifying all positive cases.*

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [14]:
#Write your code for calculating the frequency of rain here.
rain_days = len(weather1[weather1["RainToday"] == True])
total_days = len(weather1)

rain_frequency = rain_days / total_days

print("Frequency of rain in Mildura: {:.2%}".format(rain_frequency))

Frequency of rain in Mildura: 11.26%


*It rains 11.26% on average.*

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [15]:
#Write your code here
param_grid = {'kernel': ['linear',"rbf"],
              'gamma': [0.01,0.1,1.0,10.0],
              'C': [0.01,0.1,1.0,10.0]}  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_estimator_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/home/bommar70/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}
SVC(C=0.1, gamma=0.01, kernel='linear')


*The best-fit hyperparameters are:

'C': 10.0
'gamma': 0.1
'kernel': 'rbf'
*

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [16]:
grid_predictions = grid.predict(X_test)
print("Confusion matrix: \n", confusion_matrix(y_test, grid_predictions))
print(classification_report(y_test, grid_predictions, zero_division = 1))

Confusion matrix: 
 [[0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
              precision    recall  f1-score   support

        13.0       0.00      1.00      0.00         0
        15.0       1.00      0.50      0.67         2
        17.0       0.00      0.00      0.00         7
        19.0       0.00      0.00      0.00         5
        20.0       0.18      0.21      0.19        14
        22.0       0.13      0.14      0.14        14
        24.0       0.12      0.21      0.16        24
        26.0       0.04      0.04      0.04        27
        28.0       0.21      0.20      0.20        41
        30.0       0.23      0.17      0.19        36
        31.0       0.17      0.22      0.19        36
        33.0       0.12      0.11      0.11        38
        35.0       0.17      0.34      0.23        29
        37.0       0.19      0.12      0.15        24
        39.0       0.10      0.07      0.08      

## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


*Both models outperformed the baseline model, the SVM model was the most accurate. This performance is what's expected from these models.*

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


*Write down the accuracy, precision, recall, and seed number for each of your iterations.
Example:

| Random Seed | Model | Precision | Recall     |
|-------------|-------|-----------|------------|
| 3           | Logit | 0.8896    | 0.95697074 |
| 3           | SVM   | 0.89      | 0.97       |
| 2           | Logit | 0.87      | 0.94       |
| 2           | SVM   | 0.87      | 0.94       |
| 10          | Logit | 0.9       | 0.94       |
| 10          | SVM   | 0.9       | 0.95       |*

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

The precision stayed very consistant and so did the recall between both SVM and Logit which supports 5.1.

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

Both models preformed to about the same level of accuracy and precision

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University